In [1]:
#This code is for adaptive GPU usage
import keras.backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

Using TensorFlow backend.


In [2]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
from tabulate import tabulate
import datetime
import pickle as pkl
import pathlib

from keras.layers import LSTM, Dense, Input, Conv1D
from keras.models import Model
import keras.optimizers as O
import keras.losses as L
import keras.activations as A
from sklearn.metrics import accuracy_score
import numpy as np
from keras.utils import np_utils
from numpy.random import seed

# Making sure edgeml is part of python path
sys.path.insert(0, '../../')
#For processing on GPU.
#os.environ['CUDA_VISIBLE_DEVICES'] ='0'

np.random.seed(42)
tf.set_random_seed(42)

# MI-RNN and EMI-RNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_BasicLSTM, EMI_FastGRNN, EMI_FastRNN, EMI_GRU
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

In [3]:
def plot_confusion_matrix(bag_labels, bag_predictions, class_labels):
    
    import matplotlib.pyplot as plt
    import pandas as pd
    import seaborn as sn
    from sklearn.metrics import confusion_matrix
    from tabulate import tabulate

    cm = confusion_matrix(pd.DataFrame(bag_labels), pd.DataFrame(bag_predictions))
    
    cm_df = pd.DataFrame(cm,columns=class_labels, index = class_labels)
    tick_marks = np.arange(len(class_labels))
    
    plt.figure(figsize = (10,6))
    # rotation = '180' only for DSAAR.
    plt.xticks(tick_marks, class_labels, rotation=90)
    plt.yticks(tick_marks, class_labels, rotation=90)

    plt.title("Confusion Matrix")
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    
    #sns.set(rc={'figure.figsize':(10.0,10.0)})
    sn.heatmap(cm_df, annot=True, cmap = plt.cm.Blues, fmt = 'd')
    
    return

# Experiment Generator Functions

# EMI-LSTM Function

In [5]:
def lstm_experiment_generator(params, path = '/home/iot/Documents/dataset_fog_release/dataset/32_8/'):
    """
        Function that will generate the experiments to be run.
        Inputs : 
        (1) Dictionary params, to set the network parameters.
        (2) Name of the Model to be run from [EMI-LSTM, EMI-FastGRNN, EMI-GRU]
        (3) Path to the dataset, where the csv files are present.
    """
    
    #Copy the contents of the params dictionary.
    lstm_dict = {**params}
    
    #---------------------------PARAM SETTING----------------------#
    
    # Network parameters for our LSTM + FC Layer
    NUM_HIDDEN = params["NUM_HIDDEN"]
    NUM_TIMESTEPS = params["NUM_TIMESTEPS"]
    ORIGINAL_NUM_TIMESTEPS = params["ORIGINAL_NUM_TIMESTEPS"]
    NUM_FEATS = params["NUM_FEATS"]
    FORGET_BIAS = params["FORGET_BIAS"]
    NUM_OUTPUT = params["NUM_OUTPUT"]
    USE_DROPOUT = True if (params["USE_DROPOUT"] == 1) else False
    KEEP_PROB = params["KEEP_PROB"]

    # For dataset API
    PREFETCH_NUM = params["PREFETCH_NUM"]
    BATCH_SIZE = params["BATCH_SIZE"]

    # Number of epochs in *one iteration*
    NUM_EPOCHS = params["NUM_EPOCHS"]
    # Number of iterations in *one round*. After each iteration,
    # the model is dumped to disk. At the end of the current
    # round, the best model among all the dumped models in the
    # current round is picked up..
    NUM_ITER = params["NUM_ITER"]
    # A round consists of multiple training iterations and a belief
    # update step using the best model from all of these iterations
    NUM_ROUNDS = params["NUM_ROUNDS"]
    LEARNING_RATE = params["LEARNING_RATE"]

    # A staging direcory to store models
    MODEL_PREFIX = params["MODEL_PREFIX"]
    
    #----------------------END OF PARAM SETTING----------------------#
    
    #----------------------DATA LOADING------------------------------#
    
    x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
    x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
    x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

    # BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
    # step of EMI/MI RNN
    BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
    BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
    BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
    NUM_SUBINSTANCE = x_train.shape[1]
    print("x_train shape is:", x_train.shape)
    print("y_train shape is:", y_train.shape)
    print("x_test shape is:", x_val.shape)
    print("y_test shape is:", y_val.shape)
    
    #----------------------END OF DATA LOADING------------------------------#    
    
    #----------------------COMPUTATION GRAPH--------------------------------#
    
    # Define the linear secondary classifier
    def createExtendedGraph(self, baseOutput, *args, **kwargs):
        W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
        B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
        y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
        self.output = y_cap
        self.graphCreated = True

    def restoreExtendedGraph(self, graph, *args, **kwargs):
        y_cap = graph.get_tensor_by_name('y_cap_tata:0')
        self.output = y_cap
        self.graphCreated = True

    def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
        if inference is False:
            feedDict = {self._emiGraph.keep_prob: keep_prob}
        else:
            feedDict = {self._emiGraph.keep_prob: 1.0}
        return feedDict

    EMI_BasicLSTM._createExtendedGraph = createExtendedGraph
    EMI_BasicLSTM._restoreExtendedGraph = restoreExtendedGraph

    if USE_DROPOUT is True:
        EMI_Driver.feedDictFunc = feedDictFunc
    
    inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
    emiLSTM = EMI_BasicLSTM(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                            forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
    emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy',
                             stepSize=LEARNING_RATE)
    
    tf.reset_default_graph()
    g1 = tf.Graph()    
    with g1.as_default():
        # Obtain the iterators to each batch of the data
        x_batch, y_batch = inputPipeline()
        # Create the forward computation graph based on the iterators
        y_cap = emiLSTM(x_batch)
        # Create loss graphs and training routines
        emiTrainer(y_cap, y_batch)
        
    #------------------------------END OF COMPUTATION GRAPH------------------------------#
    
    #-------------------------------------EMI DRIVER-------------------------------------#
        
    with g1.as_default():
        emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

    emiDriver.initializeSession(g1)
    y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                          y_train=y_train, bag_train=BAG_TRAIN,
                                          x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                          numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                          numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                          numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                          fracEMI=0.5, updatePolicy='top-k', k=1)
    
    #-------------------------------END OF EMI DRIVER-------------------------------------#
    
    #-----------------------------------EARLY SAVINGS-------------------------------------#
    
    """
        Early Prediction Policy: We make an early prediction based on the predicted classes
        probability. If the predicted class probability > minProb at some step, we make
        a prediction at that step.
    """
    def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
        assert instanceOut.ndim == 2
        classes = np.argmax(instanceOut, axis=1)
        prob = np.max(instanceOut, axis=1)
        index = np.where(prob >= minProb)[0]
        if len(index) == 0:
            assert (len(instanceOut) - 1) == (len(classes) - 1)
            return classes[-1], len(instanceOut) - 1
        index = index[0]
        return classes[index], index

    def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
        predictionStep = predictionStep + 1
        predictionStep = np.reshape(predictionStep, -1)
        totalSteps = np.sum(predictionStep)
        maxSteps = len(predictionStep) * numTimeSteps
        savings = 1.0 - (totalSteps / maxSteps)
        if returnTotal:
            return savings, totalSteps
        return savings
    
    #--------------------------------END OF EARLY SAVINGS---------------------------------#
    
    #----------------------------------------BEST MODEL-----------------------------------#
    
    import time
    k = 2
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)
    start = time.time()
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    end = time.time()
    print(end - start)
    print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
    mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
    emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
    total_savings = mi_savings + (1 - mi_savings) * emi_savings
    print('Savings due to MI-RNN : %f' % mi_savings)
    print('Savings due to Early prediction: %f' % emi_savings)
    print('Total Savings: %f' % (total_savings))
    
    #Store in the dictionary.
    lstm_dict["k"] = k
    lstm_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
    lstm_dict["total_savings"] = total_savings
    lstm_dict["y_test"] = BAG_TEST
    lstm_dict["y_pred"] = bagPredictions
    
    # A slightly more detailed analysis method is provided. 
    df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
    print (tabulate(df, headers=list(df.columns), tablefmt='grid'))
    
    lstm_dict["detailed analysis"] = df
    #----------------------------------END OF BEST MODEL-----------------------------------#
    
    #----------------------------------PICKING THE BEST MODEL------------------------------#
    
    devnull = open(os.devnull, 'r')
    for val in modelStats:
        round_, acc, modelPrefix, globalStep = val
        
        emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)

        
        predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)
        start = time.time()
        bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)    
        end = time.time()
        print(end - start)
        print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
        print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
        print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS)) 
        
    
    #-------------------------------END OF PICKING THE BEST MODEL--------------------------#

    return lstm_dict

# Experiments 

## EMI-LSTM 

In [6]:
dataset = 'Daphn'
path = '/home/iot/Documents/dataset_fog_release/dataset/32_8/'

#Choose model from among [lstm, fastgrnn, gru]
model = 'lstm'

# Dictionary to set the parameters.
params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 32,
    "ORIGINAL_NUM_TIMESTEPS" : 50,
    "NUM_FEATS" : 9,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 3,
    "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.75,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 10,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 2,
    "LEARNING_RATE" : 0.001,
    "MODEL_PREFIX" : dataset + '/model-' + str(model)
}

#Preprocess data, and load the train,test and validation splits.
lstm_dict = lstm_experiment_generator(params, path)

#Create the directory to store the results of this run.

dirname = ""
dirname = "/home/iot/Documents" + "/"+dataset+"/"+model
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(lstm_dict,open(dirname + "/lstm_dict_" + filename + ".pkl",mode='wb'))

x_train shape is: (526, 4, 32, 9)
y_train shape is: (526, 4, 3)
x_test shape is: (59, 4, 32, 9)
y_test shape is: (59, 4, 3)
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Update policy: top-k
Training with MI-RNN loss for 1 rounds
Round: 0
Epoch   8 Batch    14 (  150) Loss 0.01057 Acc 0.84375 | Val acc 0.88194 | Model saved to Daphn/model-lstm, global_step 1000
Epoch   8 Batch    14 (  150) Loss 0.00677 Acc 0.93750 | Val acc 0.88730 | Model saved to Daphn/model-lstm, global_step 1001
Epoch   8 Batch    14 (  150) Loss 0.00501 Acc 0.93750 | Val acc 0.86169 | Model saved to Daphn/model-lstm, global_step 1002
Epoch   8 Batch    14 (  150) Loss 0.00408 Acc 0.95312 | Val acc 0.85851 | Model saved to Daphn/model-lstm, global_step 1003
INFO:tensorflow:Restoring parameters from Daphn/model-lstm-1001
Round: 1
Sw

# Fast Grnn Function

In [17]:
def fastgrnn_experiment_generator(params, path = '/home/iot/Documents/dataset_fog_release/dataset/32_8/'):
    """
        Function that will generate the experiments to be run.
        Inputs : 
        (1) Dictionary params, to set the network parameters.
        (2) Name of the Model to be run from [EMI-LSTM, EMI-FastGRNN, EMI-GRU]
        (3) Path to the dataset, where the csv files are present.
    """
    
    #Copy the params to the fastrgnn_dict.
    fastgrnn_dict = {**params}
    
    #---------------------------PARAM SETTING----------------------#
    
    # Network parameters for our FastGRNN + FC Layer
    NUM_HIDDEN = params["NUM_HIDDEN"]
    NUM_TIMESTEPS = params["NUM_TIMESTEPS"]
    NUM_FEATS = params["NUM_FEATS"]
    FORGET_BIAS = params["FORGET_BIAS"]
    NUM_OUTPUT = params["NUM_OUTPUT"]
    USE_DROPOUT = True if (params["USE_DROPOUT"] == 1) else 0
    KEEP_PROB = params["KEEP_PROB"]

    # Non-linearities can be chosen among "tanh, sigmoid, relu, quantTanh, quantSigm"
    UPDATE_NL = params["UPDATE_NL"]
    GATE_NL = params["GATE_NL"]

    # Ranks of Parameter matrices for low-rank parameterisation to compress models.
    WRANK = params["WRANK"]
    URANK = params["URANK"]

    # For dataset API
    PREFETCH_NUM = params["PREFETCH_NUM"]
    BATCH_SIZE = params["BATCH_SIZE"]

    # Number of epochs in *one iteration*
    NUM_EPOCHS = params["NUM_EPOCHS"]
    # Number of iterations in *one round*. After each iteration,
    # the model is dumped to disk. At the end of the current
    # round, the best model among all the dumped models in the
    # current round is picked up..
    NUM_ITER = params["NUM_ITER"]
    # A round consists of multiple training iterations and a belief
    # update step using the best model from all of these iterations
    NUM_ROUNDS = params["NUM_ROUNDS"]

    # A staging direcory to store models
    MODEL_PREFIX = params["MODEL_PREFIX"]
    
    #----------------------END OF PARAM SETTING----------------------#
    
    #----------------------DATA LOADING------------------------------#
    
    # Loading the data
    x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
    x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
    x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')
    
    # BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
    # step of EMI/MI RNN
    BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
    BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
    BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
    NUM_SUBINSTANCE = x_train.shape[1]
    print("x_train shape is:", x_train.shape)
    print("y_train shape is:", y_train.shape)
    print("x_test shape is:", x_val.shape)
    print("y_test shape is:", y_val.shape)
    
    #----------------------END OF DATA LOADING------------------------------#    
    
    #----------------------COMPUTATION GRAPH--------------------------------#
    
    # Define the linear secondary classifier
    def createExtendedGraph(self, baseOutput, *args, **kwargs):
        W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
        B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
        y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
        self.output = y_cap
        self.graphCreated = True

    def restoreExtendedGraph(self, graph, *args, **kwargs):
        y_cap = graph.get_tensor_by_name('y_cap_tata:0')
        self.output = y_cap
        self.graphCreated = True

    def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
        if inference is False:
            feedDict = {self._emiGraph.keep_prob: keep_prob}
        else:
            feedDict = {self._emiGraph.keep_prob: 1.0}
        return feedDict


    EMI_FastGRNN._createExtendedGraph = createExtendedGraph
    EMI_FastGRNN._restoreExtendedGraph = restoreExtendedGraph
    if USE_DROPOUT is True:
        EMI_FastGRNN.feedDictFunc = feedDictFunc
        
    inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
    emiFastGRNN = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                               gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
    emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

    tf.reset_default_graph()
    g1 = tf.Graph()    
    with g1.as_default():
        # Obtain the iterators to each batch of the data
        x_batch, y_batch = inputPipeline()
        # Create the forward computation graph based on the iterators
        y_cap = emiFastGRNN(x_batch)
        # Create loss graphs and training routines
        emiTrainer(y_cap, y_batch)
        
    #------------------------------END OF COMPUTATION GRAPH------------------------------#
    
    #-------------------------------------EMI DRIVER-------------------------------------#
        
    with g1.as_default():
        emiDriver = EMI_Driver(inputPipeline, emiFastGRNN, emiTrainer)

    emiDriver.initializeSession(g1)
    y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                          y_train=y_train, bag_train=BAG_TRAIN,
                                          x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                          numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                          numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                          numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                          fracEMI=0.5, updatePolicy='top-k', k=1)

    #-------------------------------END OF EMI DRIVER-------------------------------------#
    
    #-----------------------------------EARLY SAVINGS-------------------------------------#
    
    """
        Early Prediction Policy: We make an early prediction based on the predicted classes
        probability. If the predicted class probability > minProb at some step, we make
        a prediction at that step.
    """
    # Early Prediction Policy: We make an early prediction based on the predicted classes
    #     probability. If the predicted class probability > minProb at some step, we make
    #     a prediction at that step.
    def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
        assert instanceOut.ndim == 2
        classes = np.argmax(instanceOut, axis=1)
        prob = np.max(instanceOut, axis=1)
        index = np.where(prob >= minProb)[0]
        if len(index) == 0:
            assert (len(instanceOut) - 1) == (len(classes) - 1)
            return classes[-1], len(instanceOut) - 1
        index = index[0]
        return classes[index], index

    def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
        predictionStep = predictionStep + 1
        predictionStep = np.reshape(predictionStep, -1)
        totalSteps = np.sum(predictionStep)
        maxSteps = len(predictionStep) * numTimeSteps
        savings = 1.0 - (totalSteps / maxSteps)
        if returnTotal:
            return savings, totalSteps
        return savings
    
    #--------------------------------END OF EARLY SAVINGS---------------------------------#
    
    #----------------------------------------BEST MODEL-----------------------------------#
    
    k = 2
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb, minProb=0.99)
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
    print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS))
    
    # A slightly more detailed analysis method is provided. 
    #df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)    
    #print (tabulate(df, headers=list(df.columns), tablefmt='grid'))
    
    fastgrnn_dict["k"] = k
    fastgrnn_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
    fastgrnn_dict["additional savings"] = getEarlySaving(predictionStep, NUM_TIMESTEPS)
    #fastgrnn_dict["detailed analysis"] = df
    fastgrnn_dict["y_test"] = BAG_TEST
    fastgrnn_dict["y_pred"] = bagPredictions
    
    #----------------------------------END OF BEST MODEL-----------------------------------#
    
    
    #----------------------------------PICKING THE BEST MODEL------------------------------#
    
    devnull = open(os.devnull, 'r')
    for val in modelStats:
        round_, acc, modelPrefix, globalStep = val
        emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
        predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)

        bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
        print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
        print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
        print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS)) 
        
    
    #-------------------------------END OF PICKING THE BEST MODEL--------------------------#

    return fastgrnn_dict

# Experiment: Fast GRNN

In [58]:
dataset = 'Daphn'
path = '/home/iot/Documents/dataset_fog_release/dataset/32_8/'

#Choose model from among [lstm, fastgrnn, gru]
model = 'fastgrnn'

# Dictionary to set the parameters.
fastgrnn_params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 32,
    "NUM_FEATS" : 9,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 3,
    "USE_DROPOUT" : 0, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.9,
    "UPDATE_NL" : "quantTanh",
    "GATE_NL" : "quantSigm",
    "WRANK" : 5,
    "URANK" : 6,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 3,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 10,
    "MODEL_PREFIX" : dataset + '/model-' + str(model)
}

#Preprocess data, and load the train,test and validation splits.
fastgrnn_dict = fastgrnn_experiment_generator(fastgrnn_params, path)

#Create the directory to store the results of this run.

dirname = ""
dirname = "./Results" + ''.join(dirname) + "/"+dataset+"/"+model
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(fastgrnn_dict,open(dirname + "/fastgrnn_dict_" + filename + ".pkl",mode='wb'))

x_train shape is: (602, 4, 32, 9)
y_train shape is: (602, 4, 3)
x_test shape is: (67, 4, 32, 9)
y_test shape is: (67, 4, 3)
Update policy: top-k
Training with MI-RNN loss for 5 rounds
Round: 0


TypeError: Cannot interpret feed_dict key as Tensor: Can not convert a NoneType into a Tensor.

# EMIGRU

In [56]:
def gru_experiment_generator(params, path = '/home/iot/Documents/dataset_fog_release/dataset/32_8/'):
    """
        Function that will generate the experiments to be run.
        Inputs : 
        (1) Dictionary params, to set the network parameters.
        (2) Name of the Model to be run from [EMI-LSTM, EMI-FastGRNN, EMI-GRU]
        (3) Path to the dataset, where the csv files are present.
    """
    
    #Copy the params into the gru_dict.
    gru_dict = {**params}
    
    #---------------------------PARAM SETTING----------------------#
    
    # Network parameters for our LSTM + FC Layer
    NUM_HIDDEN = params["NUM_HIDDEN"]
    NUM_TIMESTEPS = params["NUM_TIMESTEPS"]
    ORIGINAL_NUM_TIMESTEPS = params["ORIGINAL_NUM_TIMESTEPS"]
    NUM_FEATS = params["NUM_FEATS"]
    FORGET_BIAS = params["FORGET_BIAS"]
    NUM_OUTPUT = params["NUM_OUTPUT"]
    USE_DROPOUT = True if (params["USE_DROPOUT"] == 1) else False
    KEEP_PROB = params["KEEP_PROB"]

    # For dataset API
    PREFETCH_NUM = params["PREFETCH_NUM"]
    BATCH_SIZE = params["BATCH_SIZE"]

    # Number of epochs in *one iteration*
    NUM_EPOCHS = params["NUM_EPOCHS"]
    # Number of iterations in *one round*. After each iteration,
    # the model is dumped to disk. At the end of the current
    # round, the best model among all the dumped models in the
    # current round is picked up..
    NUM_ITER = params["NUM_ITER"]
    # A round consists of multiple training iterations and a belief
    # update step using the best model from all of these iterations
    NUM_ROUNDS = params["NUM_ROUNDS"]
    LEARNING_RATE = params["LEARNING_RATE"]

    # A staging direcory to store models
    MODEL_PREFIX = params["MODEL_PREFIX"]
    
    #----------------------END OF PARAM SETTING----------------------#
    
    #----------------------DATA LOADING------------------------------#
    
    x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
    x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
    x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')
    
    # BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
    # step of EMI/MI RNN
    BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
    BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
    BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
    NUM_SUBINSTANCE = x_train.shape[1]
    print("x_train shape is:", x_train.shape)
    print("y_train shape is:", y_train.shape)
    print("x_test shape is:", x_val.shape)
    print("y_test shape is:", y_val.shape)
    
    #----------------------END OF DATA LOADING------------------------------#    
    
    #----------------------COMPUTATION GRAPH--------------------------------#
    
    # Define the linear secondary classifier
    def createExtendedGraph(self, baseOutput, *args, **kwargs):
        W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
        B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
        y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
        self.output = y_cap
        self.graphCreated = True

    def restoreExtendedGraph(self, graph, *args, **kwargs):
        y_cap = graph.get_tensor_by_name('y_cap_tata:0')
        self.output = y_cap
        self.graphCreated = True

    def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
        if inference is False:
            feedDict = {self._emiGraph.keep_prob: keep_prob}
        else:
            feedDict = {self._emiGraph.keep_prob: 1.0}
        return feedDict

    EMI_GRU._createExtendedGraph = createExtendedGraph
    EMI_GRU._restoreExtendedGraph = restoreExtendedGraph

    if USE_DROPOUT is True:
        EMI_Driver.feedDictFunc = feedDictFunc
    
    inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
    emiGRU = EMI_GRU(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                            useDropout=USE_DROPOUT)
    emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy',
                             stepSize=LEARNING_RATE)

    tf.reset_default_graph()
    g1 = tf.Graph()    
    with g1.as_default():
        # Obtain the iterators to each batch of the data
        x_batch, y_batch = inputPipeline()
        # Create the forward computation graph based on the iterators
        y_cap = emiGRU(x_batch)
        # Create loss graphs and training routines
        emiTrainer(y_cap, y_batch)
        
    #------------------------------END OF COMPUTATION GRAPH------------------------------#
    
    #-------------------------------------EMI DRIVER-------------------------------------#
        
    with g1.as_default():
        emiDriver = EMI_Driver(inputPipeline, emiGRU, emiTrainer)

    emiDriver.initializeSession(g1)
    y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                          y_train=y_train, bag_train=BAG_TRAIN,
                                          x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                          numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                          numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                          numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                          fracEMI=0.5, updatePolicy='top-k', k=1)

    #-------------------------------END OF EMI DRIVER-------------------------------------#
    
    #-----------------------------------EARLY SAVINGS-------------------------------------#
    
    """
        Early Prediction Policy: We make an early prediction based on the predicted classes
        probability. If the predicted class probability > minProb at some step, we make
        a prediction at that step.
    """
    def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
        assert instanceOut.ndim == 2
        classes = np.argmax(instanceOut, axis=1)
        prob = np.max(instanceOut, axis=1)
        index = np.where(prob >= minProb)[0]
        if len(index) == 0:
            assert (len(instanceOut) - 1) == (len(classes) - 1)
            return classes[-1], len(instanceOut) - 1
        index = index[0]
        return classes[index], index

    def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
        predictionStep = predictionStep + 1
        predictionStep = np.reshape(predictionStep, -1)
        totalSteps = np.sum(predictionStep)
        maxSteps = len(predictionStep) * numTimeSteps
        savings = 1.0 - (totalSteps / maxSteps)
        if returnTotal:
            return savings, totalSteps
        return savings
    
    #--------------------------------END OF EARLY SAVINGS---------------------------------#
    
    #----------------------------------------BEST MODEL-----------------------------------#
    
    k = 2
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
    mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
    emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
    total_savings = mi_savings + (1 - mi_savings) * emi_savings
    print('Savings due to MI-RNN : %f' % mi_savings)
    print('Savings due to Early prediction: %f' % emi_savings)
    print('Total Savings: %f' % (total_savings))
    
    # A slightly more detailed analysis method is provided. 
    df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
    print (tabulate(df, headers=list(df.columns), tablefmt='grid'))
    
    gru_dict["k"] = k
    gru_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
    gru_dict["total_savings"] = total_savings
    gru_dict["detailed analysis"] = df
    gru_dict["y_test"] = BAG_TEST
    gru_dict["y_pred"] = bagPredictions
    
    #----------------------------------END OF BEST MODEL-----------------------------------#
    
    #----------------------------------PICKING THE BEST MODEL------------------------------#
    
    devnull = open(os.devnull, 'r')
    for val in modelStats:
        round_, acc, modelPrefix, globalStep = val
        emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
        predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)

        bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
        print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
        print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
        mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
        emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
        total_savings = mi_savings + (1 - mi_savings) * emi_savings
        print("Total Savings: %f" % total_savings)
        
    #-------------------------------END OF PICKING THE BEST MODEL--------------------------#

    return gru_dict


# Experiment EMI-GRU

In [ ]:
dataset = 'Daph'
path = '/home/iot/Documents/dataset_fog_release/dataset/32_8/'

#Choose model from among [lstm, fastgrnn, gru]
model = 'gru'

# Dictionary to set the parameters.
gru_params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 32,
    "ORIGINAL_NUM_TIMESTEPS" : 50,
    "NUM_FEATS" : 9,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 3,
    "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.75,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 2,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 10,
    "LEARNING_RATE" : 0.001,
    "MODEL_PREFIX" : dataset + '/model-' + str(model)
}

#Preprocess data, and load the train,test and validation splits.
gru_dict = gru_experiment_generator(gru_params, path)

#Create the directory to store the results of this run.

dirname = ""
dirname = "/home/iot/Documents" + "/"+dataset+"/"+model
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(gru_dict,open(dirname + "/gru_dict_" + filename + ".pkl",mode='wb'))

x_train shape is: (602, 4, 32, 9)
y_train shape is: (602, 4, 3)
x_test shape is: (67, 4, 32, 9)
y_test shape is: (67, 4, 3)


Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7f4510adae10>>
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "/opt/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.InvalidArgumentError: No such callable handle: 94574852786688


Update policy: top-k
Training with MI-RNN loss for 5 rounds
Round: 0
Epoch   1 Batch    11 (   30) Loss 0.02384 Acc 0.76562 | Val acc 0.78299 | Model saved to Daph/model-gru, global_step 1000
Epoch   1 Batch    11 (   30) Loss 0.01889 Acc 0.80469 | Val acc 0.88281 | Model saved to Daph/model-gru, global_step 1001
Epoch   1 Batch    11 (   30) Loss 0.01786 Acc 0.86719 | Val acc 0.88802 | Model saved to Daph/model-gru, global_step 1002
Epoch   1 Batch    11 (   30) Loss 0.01666 Acc 0.85156 | Val acc 0.90625 | Model saved to Daph/model-gru, global_step 1003
INFO:tensorflow:Restoring parameters from Daph/model-gru-1003
Round: 1
Epoch   1 Batch    11 (   30) Loss 0.01731 Acc 0.85156 | Val acc 0.90625 | Model saved to Daph/model-gru, global_step 1004
Epoch   1 Batch    11 (   30) Loss 0.01395 Acc 0.84375 | Val acc 0.91927 | Model saved to Daph/model-gru, global_step 1005
Epoch   1 Batch    11 (   30) Loss 0.01538 Acc 0.86719 | Val acc 0.91667 | Model saved to Daph/model-gru, global_step 1006

Round:  2, Validation accuracy: 0.9349, Test Accuracy (k = 2): 0.863095, Total Savings: 0.484702
INFO:tensorflow:Restoring parameters from Daph/model-gru-1015
Round:  3, Validation accuracy: 0.9557, Test Accuracy (k = 2): 0.875000, Total Savings: 0.502381
INFO:tensorflow:Restoring parameters from Daph/model-gru-1019
Round:  4, Validation accuracy: 0.9479, Test Accuracy (k = 2): 0.875000, Total Savings: 0.506161
INFO:tensorflow:Restoring parameters from Daph/model-gru-1023
Round:  5, Validation accuracy: 0.9479, Test Accuracy (k = 2): 0.863095, Total Savings: 0.583512
INFO:tensorflow:Restoring parameters from Daph/model-gru-1026
Round:  6, Validation accuracy: 0.9557, Test Accuracy (k = 2): 0.880952, Total Savings: 0.594881
INFO:tensorflow:Restoring parameters from Daph/model-gru-1028
Round:  7, Validation accuracy: 0.9557, Test Accuracy (k = 2): 0.880952, Total Savings: 0.598780
INFO:tensorflow:Restoring parameters from Daph/model-gru-1032



# Baseline LSTM

In [68]:
def build_model(num_timesteps = 50, num_input = 9, num_hidden = 128, num_classes = 3, lr = 0.001):
    '''
        Function that builds and returns a basline LSTM model.
        Architecture : 
        Layer 1 : Input Layer.
        Layer 2 : LSTM Layer.
        Layer 3 : Dense layer with num_classes number of nodes.
        
        I/P:
            num_timesteps : no. of timesteps that are present in the dataset.
            num_input : no. of input signals for the Input layer.
            num_hidden : no. of hidden units for the LSTM layer.
            num_classes : no. of output classes, for the dense layer.
            
        Returns : 
            Constructed model.
    '''
    ip = Input(shape = (num_timesteps, num_input), name = "input")
    #x_conv_ip = Conv1D(27,2,strides = 1,padding = "same", name = "conv1")(ip)
    #x_conv_ip = Conv1D(9,2,strides = 1,padding = "same", name = "conv2")(x_conv_ip)
    #x_conv_ip = Conv1D(3,2,strides = 1,padding = "same", name = "conv3")(x_conv_ip)
    x_ip = LSTM(num_hidden, name = "lstm")(ip)
    #x_ip = Dense(32,activation = "",name = "dense1")(x_ip)
    x_op = Dense(num_classes, activation='softmax', name = "dense2")(x_ip)
    
    model = Model(inputs = [ip], outputs = [x_op])
    
    model.summary()
    
    model.compile(optimizer = O.Adam(lr = lr), loss = L.categorical_crossentropy, metrics = ['acc'])
    
    return model

def load_data(path = '/home/iot/Documents/dataset_fog_release/dataset/RAW/'):
    '''
        Function that takes in the path of a dataset and reads and returns the train, validation and test
        splits.
    '''
    x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
    x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
    x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')
    
    return x_train, y_train, x_test, y_test, x_val, y_val

def baseline_experiment_generator(params, path = "/home/iot/Documents/dataset_fog_release/dataset/RAW/"):
    '''
        Function that will run the experiments for the baseline model. The baseline model should only be run on the
        raw data, i.e. the data with the original number of timesteps.
    '''
    baseline_dict = {**params}
    
    x_train, y_train, x_test, y_test, x_val, y_val = load_data(path)
    
    model = build_model(num_timesteps = x_train.shape[1], num_input = x_train.shape[-1], num_hidden = params["num_hidden"], num_classes = y_train.shape[-1], lr = params["learning_rate"])

    model.fit(x_train,y_train,epochs = params["epochs"],batch_size = params["batch_size"],validation_data=[x_val,y_val])
    preds = model.predict(x_test,batch_size = params["batch_size"])

    #Convert the predictions into one-hot outputs.
    preds = np_utils.to_categorical(np.argmax(preds,axis=1))
    
    baseline_dict["NUM_OUTPUT"] = y_train.shape[-1]
    baseline_dict["NUM_TIMESTEPS"] = x_train.shape[1]
    baseline_dict["NUM_FEATS"] = x_train.shape[-1]
    
    baseline_dict["accuracy"] = accuracy_score(y_test,preds)    
    print ("Final Classification Accuracy : ",accuracy_score(y_test,preds))
    
    baseline_dict["y_test"] = y_test
    baseline_dict["y_pred"] = preds
    
    return baseline_dict

In [69]:
def baseline_experiment_generator(params, path = "/home/iot/Documents/dataset_fog_release/dataset/RAW/"):
    '''
        Function that will run the experiments for the baseline model. The baseline model should only be run on the
        raw data, i.e. the data with the original number of timesteps.
    '''
    baseline_dict = {**params}
    
    x_train, y_train, x_test, y_test, x_val, y_val = load_data(path)
    
    model = build_model(num_timesteps = x_train.shape[1], num_input = x_train.shape[-1], num_hidden = params["num_hidden"], num_classes = y_train.shape[-1], lr = params["learning_rate"])

    model.fit(x_train,y_train,epochs = params["epochs"],batch_size = params["batch_size"],validation_data=[x_val,y_val])
    preds = model.predict(x_test,batch_size = params["batch_size"])

    #Convert the predictions into one-hot outputs.
    preds = np_utils.to_categorical(np.argmax(preds,axis=1))
    
    baseline_dict["NUM_OUTPUT"] = y_train.shape[-1]
    baseline_dict["NUM_TIMESTEPS"] = x_train.shape[1]
    baseline_dict["NUM_FEATS"] = x_train.shape[-1]
    
    baseline_dict["accuracy"] = accuracy_score(y_test,preds)    
    print ("Final Classification Accuracy : ",accuracy_score(y_test,preds))
    
    baseline_dict["y_test"] = y_test
    baseline_dict["y_pred"] = preds
    
    return baseline_dict

In [70]:
# Baseline LSTM 

dataset = 'Daph'
model = 'baseline'

params = {
    "epochs" : 10,
    "batch_size" : 32,
    "num_hidden" : 64,
    "learning_rate" : 0.001
}

path = '/home/iot/Documents/dataset_fog_release/dataset/RAW/'

baseline_dict = baseline_experiment_generator(params, path)

#Create the directory to store the results of this run.

dirname = ""
dirname = "/home/iot/Documents/" + "/"+dataset+"/"+model
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(baseline_dict,open(dirname + "/baseline_dict_" + filename + ".pkl",mode='wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 50, 9)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                18944     
_________________________________________________________________
dense2 (Dense)               (None, 3)                 195       
Total params: 19,139
Trainable params: 19,139
Non-trainable params: 0
_________________________________________________________________
Train on 602 samples, validate on 67 samples
Epoch 1/10


InvalidArgumentError: Tensor input_2:0, specified in either feed_devices or fetch_devices was not found in the Graph